Automated script for trying to find the earliest appointments
Israel's ministry of the interior is using a web-first way to book appoinments This might be convenient- but at present the web application used has a problematic flow.

You need to enter an ID number and a phone number each time you want to change the location You then need to also instruct the system that you want to use the passport option.

The goal of this notebook is to automate the API calls and present a list of the locations + date for places with the earliest available appointments.

Current web application flow
(I don't plan to automate the first few calls)

Create an account on the myvisit.com website using a mobile phone number and click on a captcha
Enter the one time pin from the text message
Select the provider . . . (These are the steps I want to automate. )

In [8]:
import requests
import os

#to sort results- center on the coordinates of the Petah Tikva Ministry of the Interior
lat = '32.0934236'
lon = '34.8649893'

#Paste in the token taken from the browser after logging in
paste_token="13224"

#Workaround for development without leaning credentials
try:
    auth = os.environ.get('Passport_token')
except:
    auth = paste_token
    
# print(token)

In [2]:
#Parameters needed to complete the flow and auth
#OrganizationId = 56 - fixed for all passport offices LocationId - dependent on what location / office

url_get_service_type = 'https://central.qnomy.com/CentralAPI/GetServiceTypeList?organizationId=56'
r = requests.get(url_get_service_type).json()
print(r)
serviceTypeId = r['Results'][0]['serviceTypeId']
print(serviceTypeId)

{'Success': True, 'Results': [{'serviceTypeId': 156, 'serviceTypeName': 'תיאום פגישה לתיעוד ביומטרי', 'description': 'בכל פניה יינתן שירות לפונה/אדם אחד בלבד\nהנפקת דרכונים ות.ז. ביומטרים,\nשינוי מצב אישי,\nשינוי שם,\nהצהרה על תאריך לידה (יום וחודש).', 'IconPath': 'servicetypecustom/2f935c2c-47ca-401e-88af-4c0e74969d6f.png', 'availableLocationCount': 49, 'locationId': 0, 'ExtRef': '', 'Tags': ''}], 'Page': 0, 'ResultsPerPage': 0, 'TotalResults': 1, 'ErrorMessage': None, 'ErrorNumber': 0, 'Messages': None}
156


In [5]:
#get the list of locations

LocationId_list = []
LocationName_list= []
ServiceName_list= []
    

#url_get_locations = 'https://central.qnomy.com/CentralAPI/LocationSearch?currentPage=1&isFavorite=false&orderBy=Distance&organizationId=56&position=%7B%22lat%22:%2232.7882%22,%22lng%22:%2234.9595%22,%22accuracy%22:1440%7D&resultsInPage=100&serviceTypeId=156&src=mvws'
url_get_locations = f'https://central.qnomy.com/CentralAPI/LocationSearch?currentPage=1&isFavorite=false&orderBy=Distance&organizationId=56&position=%7B%22lat%22:%22{lat}%22,%22lng%22:%22{lon}%22,%22accuracy%22:1440%7D&resultsInPage=100&serviceTypeId=156&src=mvws'

r_locations = requests.get(url_get_locations).json()
# print(r_locations['Results'][0:3])

r2= requests.get(url_get_locations).json()
for a in r2['Results']:
    LocationId_list.append(a['LocationId'])
    LocationName_list.append(a['LocationName'])
    ServiceName_list.append(a['ServiceId'])
    
    #print(a['LocationName']+'       '+ str(a['LocationId'])) + "      " + str(a['ServiceId'])
print(LocationId_list)
print(LocationName_list)
print(ServiceName_list)


[809, 826, 799, 828, 802, 871, 807, 827, 821, 869, 818, 979, 819, 899, 817, 1038, 816, 1071, 859, 854, 1019, 870, 822, 1599, 825, 893, 855, 1073, 2265, 1037, 858, 849, 841, 856, 978, 861, 852, 842, 889, 860, 1822, 868, 853, 951, 866, 843, 1327, 867, 872]
['לשכת פתח תקוה', 'לשכת בני ברק', 'לשכת רמת גן-גבעתיים', 'לשכת ראש העין', 'לשכת תל אביב מרכז (קריית הממשלה)', 'לשכת הרצליה', 'לשכת כפר סבא', 'לשכת ת"א דרום (יפו)', 'לשכת חולון', 'לשכת ראשון לציון', 'לשכת רמלה', 'לשכת טייבה', 'לשכת רחובות', 'לשכת מודיעין עילית', 'לשכת נתניה', 'לשכת אריאל', 'לשכת חדרה', 'לשכת שדרות', 'לשכת בית שמש', 'לשכת אשדוד', 'לשכת קלנדיה  قلنديا', 'לשכת מזרח ירושלים شرق القدس', 'לשכת ירושלים', 'לשכת צור באהר صور باهر', 'לשכת ירושלים דרום-הר חומה', 'לשכת מעלה אדומים ', 'לשכת אשקלון', 'לשכת קרית גת', 'לשכת אום אל פחם', 'לשכת יקנעם', 'לשכת עפולה', 'לשכת רהט', 'לשכת נתיבות', 'לשכת חיפה', 'לשכת קריות (קניון שער הצפון)', ' לשכת נוף הגליל (נצרת עילית)', 'לשכת עכו', 'לשכת באר שבע', "לשכת סח'נין", 'לשכת טבריה', 'לשכת דיר אל 

In [6]:
dates = []
times = []

def get_nearest_booking(e_index):
    location_id = LocationId_list[e_index]
    location_service_code = ServiceName_list[e_index]
    location_name = LocationName_list[e_index]
    print(f"the location {location_name} - id is {location_id} and the service code is {location_service_code}")
    #get the dates
    url_get_dates =f"https://central.qnomy.com/CentralAPI/SearchAvailableDates?maxResults=10&serviceId={location_service_code}&startDate=2022-01-23"

    print(requests.get(url_get_dates, headers={'Authorization': auth}))
    r3 = requests.get(url_get_dates, headers={'Authorization': auth}).json()
    #if (len(r3['Results'])>0):
    try:
        for date in r3['Results']:
            date_code = date['calendarId']
            date_string = date['calendarDate']
            print(date_string)
            url_get_slots = f'https://central.qnomy.com/CentralAPI/SearchAvailableSlots?CalendarId={date_code}&ServiceId={location_service_code}'

            r4 = requests.get(url_get_slots, headers={'Authorization': auth}).json()
            try:
                print(r4['Results'])
    #     try:
            except:
                print(".")
    except:
        print("..")

In [ ]:
for index in range(len(LocationName_list)):
#     print(index)
    temp = get_nearest_booking(index)


In [9]:
for index in range(len(LocationName_list[:5])):
#     print(index)
    temp = get_nearest_booking(index)


the location לשכת פתח תקוה - id is 809 and the service code is 2113
<Response [200]>
2022-03-22T00:00:00
[]
2022-03-23T00:00:00
[]
2022-05-30T00:00:00
[{'Time': 576}]
2022-07-03T00:00:00
[{'Time': 744}]
2022-07-04T00:00:00
[{'Time': 612}]
2022-07-05T00:00:00
[{'Time': 624}]
2022-07-07T00:00:00
[{'Time': 564}, {'Time': 588}, {'Time': 612}, {'Time': 624}, {'Time': 636}, {'Time': 648}, {'Time': 660}, {'Time': 672}, {'Time': 684}, {'Time': 696}, {'Time': 708}, {'Time': 720}, {'Time': 732}, {'Time': 744}, {'Time': 756}, {'Time': 768}]
2022-07-10T00:00:00
[{'Time': 492}, {'Time': 504}, {'Time': 516}, {'Time': 528}, {'Time': 540}, {'Time': 552}, {'Time': 564}, {'Time': 576}, {'Time': 588}, {'Time': 600}, {'Time': 612}, {'Time': 624}, {'Time': 636}, {'Time': 648}, {'Time': 660}, {'Time': 672}, {'Time': 684}, {'Time': 696}, {'Time': 708}, {'Time': 720}, {'Time': 732}, {'Time': 744}, {'Time': 756}, {'Time': 768}]
2022-07-11T00:00:00
[{'Time': 480}, {'Time': 492}, {'Time': 504}, {'Time': 516}, {'

[{'Time': 480}, {'Time': 492}, {'Time': 504}, {'Time': 516}, {'Time': 528}, {'Time': 540}, {'Time': 552}, {'Time': 564}, {'Time': 576}, {'Time': 588}, {'Time': 600}, {'Time': 612}, {'Time': 624}, {'Time': 636}, {'Time': 648}, {'Time': 660}, {'Time': 672}, {'Time': 684}, {'Time': 696}, {'Time': 708}, {'Time': 720}, {'Time': 732}, {'Time': 744}, {'Time': 756}, {'Time': 768}, {'Time': 870}, {'Time': 882}, {'Time': 894}, {'Time': 906}, {'Time': 918}, {'Time': 930}, {'Time': 942}, {'Time': 954}, {'Time': 966}, {'Time': 978}, {'Time': 990}]
2022-07-12T00:00:00
[{'Time': 480}, {'Time': 492}, {'Time': 504}, {'Time': 516}, {'Time': 528}, {'Time': 540}, {'Time': 552}, {'Time': 564}, {'Time': 576}, {'Time': 588}, {'Time': 600}, {'Time': 612}, {'Time': 624}, {'Time': 636}, {'Time': 648}, {'Time': 660}, {'Time': 672}, {'Time': 684}, {'Time': 696}, {'Time': 708}, {'Time': 720}, {'Time': 732}, {'Time': 744}, {'Time': 756}, {'Time': 768}]
2022-07-13T00:00:00
[{'Time': 870}, {'Time': 882}, {'Time': 894